<a href="https://colab.research.google.com/github/raz0208/CityPerson_PedestrianDetection/blob/main/ModelImplementation/SSDModelForPedestrianDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instal Required Libraries
import os
import json
import numpy as np
import pandas as pd
from PIL import Image

In [ ]:
def convert_tif_to_jpg(source_dir, target_dir):
    os.makedirs(target_dir, exist_ok=True)
    for root, _, files in os.walk(source_dir):
        for file in files:
            if file.endswith('.tif'):
                tif_path = os.path.join(root, file)
                img = Image.open(tif_path).convert('L')  # Convert to grayscale if not already
                jpg_file = file.replace('.tif', '.jpg')
                target_path = os.path.join(target_dir, jpg_file)
                img.save(target_path, 'JPEG')
                #print(f"Converted {tif_path} to {target_path}")

In [ ]:
# Read the data from the source and store the result in new path
source_folder = '/kaggle/input/city-persone/gtFinePanopticParts_trainval/gtFinePanopticParts/train'
destination_folder = '/kaggle/working/CityPersonBboxPanoptic/train/images'
convert_tif_to_jpg(source_folder, destination_folder)

In [ ]:
# Read the data from the source and store the result in new path
source_folder = '/kaggle/input/city-persone/gtFinePanopticParts_trainval/gtFinePanopticParts/val'
destination_folder = '/kaggle/working/CityPersonBboxPanoptic/val/images'
convert_tif_to_jpg(source_folder, destination_folder)

In [ ]:
def convert_json_to_txt(source_dir, target_dir):
    os.makedirs(target_dir, exist_ok=True)
    for root, _, files in os.walk(source_dir):
        for file in files:
            if file.endswith('.json'):
                json_path = os.path.join(root, file)
                with open(json_path, 'r') as f:
                    data = json.load(f)

                txt_file = file.replace('.json', '.txt')
                target_path = os.path.join(target_dir, txt_file)

                with open(target_path, 'w') as f:
                    for obj in data.get('objects', []):
                        label = obj.get('label', 'unknown')
                        x, y, w, h = obj.get('bbox', [0, 0, 0, 0])
                        f.write(f"{label} {x} {y} {w} {h}\n")

                #print(f"Converted {json_path} to {target_path}")

In [ ]:
# Read the data from the source and store the result in new path
source_folder = '/kaggle/input/city-persone/gtBbox_cityPersons_trainval/gtBboxCityPersons/train'
destination_folder = '/kaggle/working/CityPersonBboxPanoptic/train/labels'
convert_json_to_txt(source_folder, destination_folder)

In [ ]:
# Read the data from the source and store the result in new path
source_folder = '/kaggle/input/city-persone/gtBbox_cityPersons_trainval/gtBboxCityPersons/val'
destination_folder = '/kaggle/working/CityPersonBboxPanoptic/val/labels'
convert_json_to_txt(source_folder, destination_folder)

In [ ]:
# Define label mapping for SSD
class_mapping = {
    "pedestrian": 0,
    "rider": 1,
    "sitting person": 2,
    "person group": 3,
    "person (other)": 4,
}

def convert_txt_to_ssd_format(source_dir, target_dir):
    os.makedirs(target_dir, exist_ok=True)
    for root, _, files in os.walk(source_dir):
        for file in files:
            if file.endswith('.txt'):
                txt_path = os.path.join(root, file)
                target_path = os.path.join(target_dir, file)

                with open(txt_path, 'r') as f:
                    lines = f.readlines()

                with open(target_path, 'w') as f:
                    for line in lines:
                        parts = line.strip().split()
                        if len(parts) == 7:
                            label, x, y, w, h, img_width, img_height = parts
                            label_id = class_mapping.get(label, -1)
                            if label_id == -1:
                                continue
                            x, y, w, h, img_width, img_height = map(int, [x, y, w, h, img_width, img_height])
                            x_min = x / img_width
                            y_min = y / img_height
                            x_max = (x + w) / img_width
                            y_max = (y + h) / img_height
                            f.write(f"{label_id} {x_min:.6f} {y_min:.6f} {x_max:.6f} {y_max:.6f}\n")

                #print(f"Converted {txt_path} to SSD format at {target_path}")

In [ ]:
# Read the data from the source and store the result in new path
source_folder = '/kaggle/working/CityPersonBboxPanoptic/train/labels'
destination_folder = '/kaggle/working/CityPersonBboxSSDFormat/train/labels'
convert_txt_to_ssd_format(source_folder, destination_folder)

In [ ]:
# Read the data from the source and store the result in new path
source_folder = '/kaggle/working/CityPersonBboxPanoptic/train/labels'
destination_folder = '/kaggle/working/CityPersonBboxSSDFormat/val/labels'
convert_txt_to_ssd_format(source_folder, destination_folder)